In [1]:
import pyspark

sc = pyspark.SparkContext()
sc

<SparkContext master=yarn appName=pyspark-shell>

In [2]:
sc.parallelize([1,2,3,4,5,6]).collect()

[1, 2, 3, 4, 5, 6]

In [10]:
def doc_to_query(line):
    line = line.split('\t')
    
    return (line[2], line[1])

def to_jsonl_entry(key, values):
    import json
    
    return json.dumps({'document_id': key, 'queries': [i for i in values]})

sc.textFile('file:///mnt/ceph/storage/corpora/corpora-thirdparty/corpus-orcas/orcas.tsv.gz')\
    .map(doc_to_query)\
    .repartition(100)\
    .groupByKey()\
    .map(lambda i: to_jsonl_entry(i[0], i[1]))\
    .saveAsTextFile('ecir2022/anchor2query/orcas-grouped-by-document.jsonl')

In [3]:
import json
tmp_doc_to_orcas = sc.textFile('ecir2022/anchor2query/orcas-grouped-by-document.jsonl').map(lambda i: json.loads(i)).collect()

len(tmp_doc_to_orcas)

1422029

In [4]:
from tqdm import tqdm
doc_to_orcas = {}

for i in tqdm(tmp_doc_to_orcas):
    doc_to_orcas[i['document_id']]= i['queries']

len(doc_to_orcas)

100%|██████████| 1422029/1422029 [00:01<00:00, 828873.72it/s]


1422029

In [5]:
def map_to_orcas(i):
    import json
    i = json.loads(i)
    orcasQueries = []
    for targetDocId in i['targetMsMarcoDocIds']:
        if targetDocId in doc_to_orcas:
            new_queries = doc_to_orcas[targetDocId]
            if len(new_queries) > 0:
                orcasQueries += new_queries
    
    if len(orcasQueries) <= 0:
        return None
    
    i['orcasQueries'] = orcasQueries
    return i

In [7]:
sc.textFile('../anjyc/script-filtered-commoncrawl-2019-47-repartitioned/script-commoncrawl-2019-47-PART-*.jsonl/part-*.gz')\
    .map(map_to_orcas)\
    .filter(lambda i: i is not None)\
    .map(lambda i: json.dumps(i))\
    .saveAsTextFile('ecir2022/anchor2query/anchor-text-with-orcas-queries-2019-47.jsonl')

In [20]:
def queries(doc_ids):
    import json
    ret = []
    with open('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir21-anchor2query/document-id-to-orcas-queries.jsonl', 'r') as f:
        for l in f:
            l = json.loads(l)
            if l['documentId'] in doc_ids:
                ret += l['orcasQueries']
    return ret

def map_to_orcas(i):
    import json
    i = json.loads(i)
    orcasQueries = queries(i['targetMsMarcoDocIds'])
    if len(orcasQueries) <= 0:
        return None
    
    i['orcasQueries'] = orcasQueries
    return i

In [10]:
with open('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir21-anchor2query/document-id-to-orcas-queries.jsonl', 'w') as f:
    for k,v in tqdm(doc_to_orcas.items()):
        f.write(json.dumps({'documentId': k, 'orcasQueries': [i for i in v]}) + '\n')


100%|██████████| 1422029/1422029 [00:21<00:00, 65533.39it/s]


In [19]:
%%time
queries(['D2595172'])

CPU times: user 12.1 s, sys: 567 ms, total: 12.7 s
Wall time: 12.7 s


['actress sondra locke',
 'gordon anderson actor',
 'how old is sandra locke',
 'sonia locke',
 'sondra locke children with clint eastwood',
 'sondra locke clint eastwood',
 'sondra locke death',
 'sondra locke died',
 'sondra locke films',
 'sondra locke hot',
 'sondra locke husband',
 'sondra locke images',
 'sudden impact',
 'sondra locke book',
 'sondra locke cancer',
 'sondra locke cause of death',
 'sondra locke children',
 "clint eastwood's girlfriend sandra",
 'whatever happened to sondra locke',
 'sondra loche',
 'sondra lock',
 'gordon anderson and sondra locke',
 'gordon anderson sculptor',
 "gordon anderson sondra locke's husband",
 'locke sondra',
 'sonda locke',
 'sonya locke',
 'wiki sondra locke',
 'sondra locke',
 'sondra locke 2017',
 'recent pictures of sondra locke',
 'clint eastwood hometown',
 'sondralocke',
 'sondre locke',
 'scott cunneen and sondra locke',
 'sondra locke wikipedia',
 'sondra locke young',
 "sondra locke's brother don locke",
 "sondra locke's da

In [16]:
dir(json)

['JSONDecodeError',
 'JSONDecoder',
 'JSONEncoder',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_default_decoder',
 '_default_encoder',
 'codecs',
 'decoder',
 'detect_encoding',
 'dump',
 'dumps',
 'encoder',
 'load',
 'loads',
 'scanner']